In [6]:
import pandas as pd
import cudf
%cd /data/dhk/physionet.org/files/mimiciv/2.2/hosp

/data/dhk/physionet.org/files/mimiciv/2.2/hosp


In [7]:
# proodh = pd.read_csv("process_dohun.csv")
df_dh = cudf.read_csv("process_dohun.csv")

df_dh = df_dh.drop('subject_id', axis=1)
df_dh = df_dh.drop('hadm_id', axis=1)
df_dh

,admit_age,gender,insurance,language,marital_status,race,hospital_expire_flag,medication,frequency,drug
0,-0.928571,1,2,1,1,28,0,50,95,34
1,-0.071429,0,2,1,1,28,0,806,16,860
2,-2.071429,0,2,1,1,28,0,50,95,72
3,-0.571429,1,2,1,2,28,0,15,16,1786
4,-1.000000,1,2,1,0,28,0,458,91,503
...,...,...,...,...,...,...,...,...,...,...
10350291,-0.785714,1,2,1,2,28,1,1552,121,1659
10350292,-0.785714,1,2,1,2,28,1,1552,121,1659
10350293,-0.785714,1,2,1,2,28,1,1552,121,1659
10350294,-0.785714,1,2,1,2,28,1,1552,121,1659


In [8]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from cuml.model_selection import train_test_split  


model = CatBoostClassifier(loss_function='Logloss')

y = df_dh['hospital_expire_flag'] 
X = df_dh.drop('hospital_expire_flag', axis=1) # 'label' 열을 제외한 나머지 열을 X로 사용


X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8,random_state=42)

X_train = X_train.to_cupy()
X_test = X_test.to_cupy()
y_train = y_train.to_cupy()
y_test = y_test.to_cupy()

X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

# X_train = X_train.to_numpy()
# X_test = X_test.to_numpy()
# y_train = y_train.to_numpy()
# y_test = y_test.to_numpy()

# 매개변수 그리드 정의
param = {
  'learning_rate': [0.1, 0.01],
  'depth': [3, 4, 5],
  'iterations': [100, 200],
  'l2_leaf_reg': [1, 3],
  'random_strength': [0.1, 0.5]
}
# GridSearchCV 객체 생성
grid_search = GridSearchCV(model, param_grid=param, cv=3, refit=True, n_jobs=-1, return_train_score=True, verbose=2, error_score='raise')
# 훈련 데이터로 모델 학습 및 최적 매개변수 탐색
grid_search.fit(X_train.get(), y_train.get())
# 최적 매개변수 출력
print("최적 매개변수:", grid_search.best_params_)
# 최적 매개변수로 모델 재학습
best_model = grid_search.best_estimator_
# best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)

Fitting 3 folds for each of 48 candidates, totalling 144 fits
0:	learn: 0.6924708	total: 4.05s	remaining: 6m 41s
1:	learn: 0.6918096	total: 7.37s	remaining: 6m 1s
2:	learn: 0.6913563	total: 11s	remaining: 5m 56s
3:	learn: 0.6904120	total: 14.9s	remaining: 5m 58s
4:	learn: 0.6900245	total: 18.2s	remaining: 5m 46s
5:	learn: 0.6894135	total: 21.7s	remaining: 5m 40s
6:	learn: 0.6888879	total: 24.8s	remaining: 5m 28s
7:	learn: 0.6883356	total: 26.9s	remaining: 5m 9s
8:	learn: 0.6877528	total: 30.5s	remaining: 5m 7s
9:	learn: 0.6872022	total: 33.5s	remaining: 5m 1s
10:	learn: 0.6865522	total: 36.2s	remaining: 4m 52s
11:	learn: 0.6860681	total: 38.3s	remaining: 4m 41s
12:	learn: 0.6855990	total: 41.5s	remaining: 4m 37s
13:	learn: 0.6850994	total: 44.2s	remaining: 4m 31s
14:	learn: 0.6844927	total: 47.1s	remaining: 4m 27s
15:	learn: 0.6840876	total: 50s	remaining: 4m 22s
16:	learn: 0.6835921	total: 53.3s	remaining: 4m 20s
17:	learn: 0.6831294	total: 56.3s	remaining: 4m 16s
18:	learn: 0.6826463

/home/dhkim/anaconda3/envs/mach/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


55:	learn: 0.5581574	total: 2m 46s	remaining: 2m 10s
56:	learn: 0.5568123	total: 2m 49s	remaining: 2m 7s
57:	learn: 0.5562346	total: 2m 52s	remaining: 2m 4s
58:	learn: 0.5540481	total: 2m 55s	remaining: 2m 1s
59:	learn: 0.5532607	total: 2m 58s	remaining: 1m 59s
60:	learn: 0.5525190	total: 3m 1s	remaining: 1m 56s
61:	learn: 0.5514404	total: 3m 4s	remaining: 1m 52s
62:	learn: 0.5496101	total: 3m 7s	remaining: 1m 49s
63:	learn: 0.5479266	total: 3m 10s	remaining: 1m 47s
64:	learn: 0.5466404	total: 3m 14s	remaining: 1m 44s
65:	learn: 0.5459346	total: 3m 17s	remaining: 1m 41s
66:	learn: 0.5440777	total: 3m 20s	remaining: 1m 38s
67:	learn: 0.5427614	total: 3m 23s	remaining: 1m 35s
68:	learn: 0.5421305	total: 3m 27s	remaining: 1m 33s
69:	learn: 0.5400083	total: 3m 30s	remaining: 1m 30s
70:	learn: 0.5386524	total: 3m 34s	remaining: 1m 27s
71:	learn: 0.5376079	total: 3m 38s	remaining: 1m 25s
72:	learn: 0.5359173	total: 3m 42s	remaining: 1m 22s
73:	learn: 0.5336096	total: 3m 45s	remaining: 1m 19s

CatBoostError: Invalid data type=<class 'cupy.ndarray'> : must be list, numpy.ndarray, pandas.Series, pandas.DataFrame, scipy.sparse matrix, catboost.FeaturesData or catboost.Pool

153:	learn: 0.3636007	total: 12m 7s	remaining: 3m 37s
154:	learn: 0.3634563	total: 12m 12s	remaining: 3m 32s
155:	learn: 0.3627361	total: 12m 17s	remaining: 3m 27s
156:	learn: 0.3619630	total: 12m 21s	remaining: 3m 23s
157:	learn: 0.3614377	total: 12m 24s	remaining: 3m 17s
158:	learn: 0.3610314	total: 12m 28s	remaining: 3m 12s
159:	learn: 0.3602624	total: 12m 31s	remaining: 3m 7s
160:	learn: 0.3596908	total: 12m 35s	remaining: 3m 2s
161:	learn: 0.3592117	total: 12m 39s	remaining: 2m 58s
162:	learn: 0.3585035	total: 12m 43s	remaining: 2m 53s
163:	learn: 0.3577468	total: 12m 48s	remaining: 2m 48s
164:	learn: 0.3574466	total: 12m 51s	remaining: 2m 43s
165:	learn: 0.3572885	total: 12m 54s	remaining: 2m 38s
166:	learn: 0.3565346	total: 12m 59s	remaining: 2m 34s
167:	learn: 0.3562475	total: 13m 3s	remaining: 2m 29s
168:	learn: 0.3555348	total: 13m 8s	remaining: 2m 24s
169:	learn: 0.3551290	total: 13m 12s	remaining: 2m 19s
170:	learn: 0.3549300	total: 13m 16s	remaining: 2m 15s
171:	learn: 0.3

In [9]:
y_pred = best_model.predict(X_test.get())

In [10]:
import cuml
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

cu_score = cuml.metrics.accuracy_score( y_test, y_pred )
print(cu_score)

# 정확도 계산
accuracy = accuracy_score(y_test.get(), y_pred)
print("Accuracy:", accuracy)

# F1 스코어 계산
f1 = f1_score(y_test.get(), y_pred)
print("F1 Score:", f1)

0.8688207268714905
Accuracy: 0.8688207105108064
F1 Score: 0.873493905742713


In [11]:
import pandas as pd

# randomForest_y_pred를 Pandas DataFrame으로 변환
y_pred_df = pd.DataFrame(y_pred)

# CSV 파일로 저장
y_pred_df.to_csv('catboost_y_pred.csv', index=False, header=False)